# Omission LFP Analysis

Brief 1-2 sentence description of notebook.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import glob
from collections import defaultdict
import re

In [3]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from scipy import stats
import itertools

In [4]:
import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp
from spectral_connectivity import Multitaper, Connectivity

## Inputs & Data

Explanation of each input and where it comes from.

In [5]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case

In [6]:
BBOX_TO_ANCHOR=(1.5, 0.9)
LOC='upper right'

In [7]:
# variables for LFP extraction
FREQ_MIN=0.5
FREQ_MAX=300
FREQ=60
RESAMPLE_RATE=1000
TRIAL_DURATION=10

In [8]:
INPUT_VARIABLE = 1

TIME_HALFBANDWIDTH_PRODUCT = 2
TIME_WINDOW_DURATION = 3
TIME_WINDOW_STEP = 1.5 

TRIAL_TIME_STAMP_DURATION = 1000*10

In [9]:
BIN_TO_COLOR = {0: {"baseline": "lightblue", "trial": "blue"}, 1: {"baseline": "lightgreen", "trial": "green"}, 2: {"baseline": "lightcoral", "trial": "red"}}
TRIAL_OR_BASELINE_TO_STYLE = {'baseline': "--", "trial": "-"}
BIN_TO_VELOCITY = {0: "0 to 2.5cm/s", 1: "2.5 to 5cm/s", 2: "5cm/s to 10cm/s"}

In [10]:
BIN_TO_COLOR = {0: {"baseline": "lightblue", "trial": "blue"}, 1: {"baseline": "lightgreen", "trial": "green"}, 2: {"baseline": "lightcoral", "trial": "red"}}
TRIAL_OR_BASELINE_TO_STYLE = {'baseline': "--", "trial": "-"}
BIN_TO_VELOCITY = {0: "0 to 5cm/s", 1: "5cm/s to 10cm/s", 2: "10cm/s+"}

In [11]:
NUM_LINES = 3

In [12]:
# Generate colors from the "Blues" colormap
LOSING_COLORS = cm.Oranges(np.linspace(0.5, 1, NUM_LINES))
# Generate colors from the "Blues" colormap
WINNING_COLORS = cm.Blues(np.linspace(0.5, 1, NUM_LINES))
# Generate colors from the "Blues" colormap
REWARDED_COLORS = cm.Greens(np.linspace(0.5, 1, NUM_LINES))
# Generate colors from the "Blues" colormap
OMISSION_COLORS = cm.Reds(np.linspace(0.5, 1, NUM_LINES))

In [13]:
BASELINE_OUTCOME_TO_COLOR = {'lose': "orange",
 'lose_baseline': LOSING_COLORS[0],
 'omission': "red",
 'omission_baseline': "hotpink",
 'rewarded': "green",
 'rewarded_baseline': REWARDED_COLORS[0],
 'win': "blue",
 'win_baseline': WINNING_COLORS[0]}

In [14]:
COMPETITIVE_OUTCOME_TO_COLOR = {'lose_comp': "orange", 
'lose_non_comp': "yellow",
'omission': "red",
'rewarded': "green",
'win_comp': "blue", 
'win_non_comp': WINNING_COLORS[0]}

In [15]:
TRIAL_OR_BASELINE_TO_STYLE = {'baseline': "--", "trial": "-"}

In [16]:
TONE_TIMESTAMP_DF = pd.read_excel("./proc/melted_trials_and_video.pkl", index_col=0)
OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs


ValueError: Excel file format cannot be determined, you must specify an engine manually.

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions

- Ideally functions are defined here first and then data is processed using the functions
    - function names are short and in snake case all lowercase
    - a function name should be unique but does not have to describe the function
    - doc strings describe functions not function names

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [ ]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

In [ ]:
raise ValueError()

## Power Calculation

In [ ]:
all_trials_df.head()

- Adding all the brain region to ch information

In [ ]:
channel_map_and_all_trials_df = all_trials_df.merge(CHANNEL_MAPPING_DF, left_on="current_subject", right_on="Subject", how="left")

- Linking up all LFP calculations with all the trials

In [ ]:
channel_map_and_all_trials_df["all_ch_lfp"] = channel_map_and_all_trials_df["recording_file"].map(recording_name_to_all_ch_lfp)

- Creating a new row for each brain region

In [ ]:
brain_region_col = [col for col in CHANNEL_MAPPING_DF if "spike_interface" in col]

In [ ]:
id_cols = [col for col in channel_map_and_all_trials_df.columns if col not in brain_region_col]

In [ ]:
melted_channel_map_and_all_trials_df = channel_map_and_all_trials_df.melt(id_vars=id_cols ,value_vars=brain_region_col, var_name='brain_region', value_name='channel')


In [ ]:
melted_channel_map_and_all_trials_df.head()

In [ ]:
melted_channel_map_and_all_trials_df.tail()

- Getting the traces for each trial for each brain region

In [ ]:
melted_channel_map_and_all_trials_df["channel"] = melted_channel_map_and_all_trials_df["channel"].astype(int).astype(str)
melted_channel_map_and_all_trials_df["resampled_index"] = melted_channel_map_and_all_trials_df["resampled_index"].astype(int)

In [ ]:
melted_channel_map_and_all_trials_df["chunked_lfp_index"].iloc[0]

In [ ]:
melted_channel_map_and_all_trials_df["binned_current_thorax_velocity"].iloc[0]

In [ ]:
melted_channel_map_and_all_trials_df["chunked_current_trace"] = melted_channel_map_and_all_trials_df.apply(lambda x: 
[x["all_ch_lfp"].get_traces(channel_ids=[x["channel"]], start_frame=chunk_index[0], end_frame=chunk_index[1]).T[0] for chunk_index in x["chunked_lfp_index"]], axis=1)

In [ ]:
len(melted_channel_map_and_all_trials_df["chunked_current_trace"].iloc[0])

In [ ]:
melted_channel_map_and_all_trials_df["chunked_current_trace"]

In [ ]:
melted_channel_map_and_all_trials_df["chunk_id"] = [list(range(0,10))] * len(melted_channel_map_and_all_trials_df)

In [ ]:
exploded_channel_map_and_all_trials_df = melted_channel_map_and_all_trials_df.explode(["current_chunked_thorax_velocity", "chunked_current_trace", "binned_current_thorax_velocity", "chunked_lfp_index",	"chunked_ephys_timestamp", "chunk_id"])

In [ ]:
exploded_channel_map_and_all_trials_df

In [ ]:
exploded_channel_map_and_all_trials_df["len_chunked_current_trace"] = exploded_channel_map_and_all_trials_df["chunked_current_trace"].apply(lambda x: len(x))


In [ ]:
exploded_channel_map_and_all_trials_df["len_chunked_current_trace"].unique()

In [ ]:
exploded_channel_map_and_all_trials_df["chunked_current_trace"] = exploded_channel_map_and_all_trials_df["chunked_current_trace"].apply(lambda x: x[:RESAMPLE_RATE])

In [ ]:
exploded_channel_map_and_all_trials_df["chunked_current_trace"]

- Calcuating the power at each frequency band

In [ ]:
exploded_channel_map_and_all_trials_df["multitaper"] = exploded_channel_map_and_all_trials_df["chunked_current_trace"].apply(lambda x: Multitaper(time_series=x,                                                                                        sampling_frequency=RESAMPLE_RATE))

In [ ]:
exploded_channel_map_and_all_trials_df["connectivity"] = exploded_channel_map_and_all_trials_df["multitaper"].apply(lambda x: Connectivity.from_multitaper(x))

In [ ]:
exploded_channel_map_and_all_trials_df["frequencies"] = exploded_channel_map_and_all_trials_df["connectivity"].apply(lambda x: x.frequencies)

In [ ]:
exploded_channel_map_and_all_trials_df["power"] = exploded_channel_map_and_all_trials_df["connectivity"].apply(lambda x: x.power().squeeze())

In [ ]:
exploded_channel_map_and_all_trials_df.head()

In [ ]:
exploded_channel_map_and_all_trials_df.tail()

# Plotting by velocity

- Grouping all the rows(1s segment of trials) that share the same brain region, whether its before or after the tone, and velocity grouping

In [ ]:
grouped_all_trials_df = exploded_channel_map_and_all_trials_df.groupby(['brain_region', 'trial_or_baseline', 'binned_current_thorax_velocity']).agg({'power': lambda x: np.vstack(x.tolist())}).reset_index()

- Calculating the mean and sem of the LFP power

In [ ]:
grouped_all_trials_df["mean_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanmean(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["std_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanstd(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["n_trials"] = grouped_all_trials_df["power"].apply(lambda x: np.sum(~np.isnan(x), axis=0))

In [ ]:
grouped_all_trials_df["sem_power"] = grouped_all_trials_df.apply(lambda x: x["std_power"] / np.sqrt(x["n_trials"]), axis=1)

In [ ]:
low_freq = 0
high_freq = 12
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]

    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {}".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.01, 0.1)

    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} for {}".format(BIN_TO_VELOCITY[row["binned_current_thorax_velocity"]], \
            row["trial_or_baseline"]), color=BIN_TO_COLOR[row["binned_current_thorax_velocity"]][row["trial_or_baseline"]],
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)

            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], \
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], alpha=0.2,
            color=BIN_TO_COLOR[row["binned_current_thorax_velocity"]][row["trial_or_baseline"]])
        except Exception as e: 
            print(e)
            continue

    plt.legend(loc="lower left")
    
    plt.savefig("./proc/velocity_parsing/plot_region_line_velocity_baselinetrial/\
    rce_lfp_power_velocity_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))


In [ ]:
low_freq = 12
high_freq = 30
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]

    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {}".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.002, 0.015) 

    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} for {}".format(BIN_TO_VELOCITY[row["binned_current_thorax_velocity"]], \
            row["trial_or_baseline"]), color=BIN_TO_COLOR[row["binned_current_thorax_velocity"]][row["trial_or_baseline"]],
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)

            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], \
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], alpha=0.2,
            color=BIN_TO_COLOR[row["binned_current_thorax_velocity"]][row["trial_or_baseline"]])
        except Exception as e: 
            print(e)
            continue

    plt.savefig("./proc/velocity_parsing/plot_region_line_velocity_baselinetrial/\
    rce_lfp_power_velocity_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))


In [ ]:
low_freq = 30
high_freq = 90
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]

    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {}".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    # plt.ylim(0, 0.005)
    plt.yscale("log")
    plt.ylim(0.0003, 0.005) 

    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} for {}".format(BIN_TO_VELOCITY[row["binned_current_thorax_velocity"]], \
            row["trial_or_baseline"]), color=BIN_TO_COLOR[row["binned_current_thorax_velocity"]][row["trial_or_baseline"]],
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)

            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], \
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], alpha=0.2,
            color=BIN_TO_COLOR[row["binned_current_thorax_velocity"]][row["trial_or_baseline"]])
        except Exception as e: 
            print(e)
            continue
    plt.legend()
    plt.savefig("./proc/velocity_parsing/plot_region_line_velocity_baselinetrial/\
    rce_lfp_power_velocity_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))


## Plotting all the lower velocities together

In [ ]:
speed_filtered_all_trials_df = exploded_channel_map_and_all_trials_df[exploded_channel_map_and_all_trials_df["binned_current_thorax_velocity"] == 0]

In [ ]:
speed_filtered_all_trials_df.shape

In [ ]:
grouped_all_trials_df = speed_filtered_all_trials_df.groupby(['brain_region', 'trial_outcome', 'trial_or_baseline']).agg({'power': lambda x: np.vstack(x.tolist())}).reset_index()

In [ ]:
grouped_all_trials_df["mean_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanmean(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["std_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanstd(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["n_trials"] = grouped_all_trials_df["power"].apply(lambda x: np.sum(~np.isnan(x), axis=0))

In [ ]:
grouped_all_trials_df["sem_power"] = grouped_all_trials_df.apply(lambda x: x["std_power"] / np.sqrt(x["n_trials"]), axis=1)

In [ ]:
grouped_all_trials_df["trial_outcome"].unique()

# Drawing plots all together

In [ ]:
low_freq = 0
high_freq = 12
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 0 to 5cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq) 
    # plt.yscale("log")
    # plt.ylim(0.01, 0.06)

    for trial_or_baseline, outcome in itertools.product(region_df["trial_or_baseline"].unique(), sorted(region_df["trial_outcome"].unique())[::-1]):
        outcome_df = region_df[(region_df["trial_or_baseline"] == trial_or_baseline) & (region_df["trial_outcome"] == outcome)]
        for index, row in outcome_df.iterrows():
            try:
                ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
                label="{} {}".format(outcome, trial_or_baseline), color=BASELINE_OUTCOME_TO_COLOR[outcome], \
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[trial_or_baseline], linewidth=3)
                
                plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
                row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
                alpha=0.1, color=BASELINE_OUTCOME_TO_COLOR[outcome])
            
            except Exception as e: 
                print(e)
                continue
    plt.legend(loc="lower left")
    plt.savefig("./proc/velocity_parsing/plot_region_line_trialoutcome_baselinetrial/\
    rce_lower_velocity_lfp_power_trialoutcome_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))

In [ ]:
low_freq = 12
high_freq = 30
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 0 to 5cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.002, 0.015) 
    
    for trial_or_baseline, outcome in itertools.product(region_df["trial_or_baseline"].unique(), sorted(region_df["trial_outcome"].unique())[::-1]):
        outcome_df = region_df[(region_df["trial_or_baseline"] == trial_or_baseline) & (region_df["trial_outcome"] == outcome)]
        for index, row in outcome_df.iterrows():
            try:
                ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
                label="{} {}".format(outcome, trial_or_baseline), color=BASELINE_OUTCOME_TO_COLOR[outcome], \
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[trial_or_baseline], linewidth=3)
                
                plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
                row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
                alpha=0.1, color=BASELINE_OUTCOME_TO_COLOR[outcome])
            
            except Exception as e: 
                print(e)
                continue
    plt.legend(loc="upper right")
    plt.savefig("./proc/velocity_parsing/plot_region_line_trialoutcome_baselinetrial/\
    rce_lower_velocity_lfp_power_trialoutcome_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))

In [ ]:
low_freq = 30
high_freq = 90
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 0 to 5cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.0003, 0.005) 
    
    for trial_or_baseline, outcome in itertools.product(region_df["trial_or_baseline"].unique(), sorted(region_df["trial_outcome"].unique())[::-1]):
        outcome_df = region_df[(region_df["trial_or_baseline"] == trial_or_baseline) & (region_df["trial_outcome"] == outcome)]
        for index, row in outcome_df.iterrows():
            try:
                ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
                label="{} {}".format(outcome, trial_or_baseline), color=BASELINE_OUTCOME_TO_COLOR[outcome], \
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[trial_or_baseline], linewidth=3)
                
                plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
                row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
                alpha=0.2, color=BASELINE_OUTCOME_TO_COLOR[outcome])
            
            except Exception as e: 
                print(e)
                continue
    plt.legend(loc="upper right")
    plt.savefig("./proc/velocity_parsing/plot_region_line_trialoutcome_baselinetrial/\
    rce_lower_velocity_lfp_power_trialoutcome_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))

## Plotting all the higher velocities together

In [ ]:
speed_filtered_all_trials_df = exploded_channel_map_and_all_trials_df[exploded_channel_map_and_all_trials_df["binned_current_thorax_velocity"] == 1]

In [ ]:
exploded_channel_map_and_all_trials_df["theta_power"] = exploded_channel_map_and_all_trials_df["power"].apply(lambda x: x[4:13])

In [ ]:
exploded_channel_map_and_all_trials_df["theta_power"].iloc[0]

In [ ]:
exploded_channel_map_and_all_trials_df["averaged_theta_power"] = exploded_channel_map_and_all_trials_df["theta_power"].apply(lambda x: np.mean(x))

In [ ]:
exploded_channel_map_and_all_trials_df["averaged_theta_power"]

In [ ]:
exploded_channel_map_and_all_trials_df.columns

In [ ]:
exploded_channel_map_and_all_trials_df["all_thorax_velocity"].iloc[0].shape

In [ ]:
exploded_channel_map_and_all_trials_df.info()

In [ ]:
exploded_channel_map_and_all_trials_df.drop(columns=["multitaper", "connectivity", 'all_thorax_locations', 'all_thorax_velocity', 'all_ch_lfp']).to_pickle("./proc/rce_1_sec_interval_lfp_power_spectra_and_velocity.pkl")

In [ ]:
grouped_all_trials_df = speed_filtered_all_trials_df.groupby(['brain_region', 'trial_outcome', 'trial_or_baseline']).agg({'power': lambda x: np.vstack(x.tolist())}).reset_index()

In [ ]:
grouped_all_trials_df["mean_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanmean(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["std_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanstd(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["n_trials"] = grouped_all_trials_df["power"].apply(lambda x: np.sum(~np.isnan(x), axis=0))

In [ ]:
grouped_all_trials_df["sem_power"] = grouped_all_trials_df.apply(lambda x: x["std_power"] / np.sqrt(x["n_trials"]), axis=1)

In [ ]:
exploded_channel_map_and_all_trials_df["frequencies"].iloc[0][4:13]

In [ ]:
grouped_all_trials_df["power"].iloc[0].shape

In [ ]:
grouped_all_trials_df["power"].iloc

# Drawing plots all together

In [ ]:
low_freq = 0
high_freq = 12
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 5cm/s to 10cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)   
    plt.yscale("log")
    plt.ylim(0.01, 0.1)

    for trial_or_baseline, outcome in itertools.product(region_df["trial_or_baseline"].unique(), sorted(region_df["trial_outcome"].unique())[::-1]):
        outcome_df = region_df[(region_df["trial_or_baseline"] == trial_or_baseline) & (region_df["trial_outcome"] == outcome)]
        for index, row in outcome_df.iterrows():
            try:
                ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
                label="{} {}".format(outcome, trial_or_baseline), color=BASELINE_OUTCOME_TO_COLOR[outcome], \
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[trial_or_baseline], linewidth=3)
                
                plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
                row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
                alpha=0.2, color=BASELINE_OUTCOME_TO_COLOR[outcome])
            
            except Exception as e: 
                print(e)
                continue
    plt.legend()
    plt.savefig("./proc/velocity_parsing/plot_region_line_trialoutcome_baselinetrial/\
    rce_higher_velocity_lfp_power_trialoutcome_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))

In [ ]:
low_freq = 12
high_freq = 30
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 5cm/s to 10cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.002, 0.015) 
    
    for trial_or_baseline, outcome in itertools.product(region_df["trial_or_baseline"].unique(), sorted(region_df["trial_outcome"].unique())[::-1]):
        outcome_df = region_df[(region_df["trial_or_baseline"] == trial_or_baseline) & (region_df["trial_outcome"] == outcome)]
        for index, row in outcome_df.iterrows():
            try:
                ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
                label="{} {}".format(outcome, trial_or_baseline), color=BASELINE_OUTCOME_TO_COLOR[outcome], \
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[trial_or_baseline], linewidth=3)
                
                plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
                row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
                alpha=0.2, color=BASELINE_OUTCOME_TO_COLOR[outcome])
            
            except Exception as e: 
                print(e)
                continue
    plt.legend()
    plt.savefig("./proc/velocity_parsing/plot_region_line_trialoutcome_baselinetrial/\
    rce_higher_velocity_lfp_power_trialoutcome_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))

In [ ]:
low_freq = 30
high_freq = 90
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 5cm/s to 10cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.0003, 0.008) 
    
    for trial_or_baseline, outcome in itertools.product(region_df["trial_or_baseline"].unique(), sorted(region_df["trial_outcome"].unique())[::-1]):
        outcome_df = region_df[(region_df["trial_or_baseline"] == trial_or_baseline) & (region_df["trial_outcome"] == outcome)]
        for index, row in outcome_df.iterrows():
            try:
                ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
                label="{} {}".format(outcome, trial_or_baseline), color=BASELINE_OUTCOME_TO_COLOR[outcome], \
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[trial_or_baseline], linewidth=3)
                
                plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
                row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
                alpha=0.2, color=BASELINE_OUTCOME_TO_COLOR[outcome])
            
            except Exception as e: 
                print(e)
                continue
    plt.legend()
    plt.savefig("./proc/velocity_parsing/plot_region_line_trialoutcome_baselinetrial/\
    rce_higher_velocity_lfp_power_trialoutcome_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))

## Plotting all the competitiveness and lower velocities together

In [ ]:
speed_filtered_all_trials_df = exploded_channel_map_and_all_trials_df[exploded_channel_map_and_all_trials_df["binned_current_thorax_velocity"] == 0]

In [ ]:
speed_filtered_all_trials_df.shape

In [ ]:
grouped_all_trials_df = speed_filtered_all_trials_df.groupby(['brain_region', 'competition_closeness', 'trial_or_baseline']).agg({'power': lambda x: np.vstack(x.tolist())}).reset_index()

In [ ]:
grouped_all_trials_df["mean_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanmean(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["std_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanstd(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["n_trials"] = grouped_all_trials_df["power"].apply(lambda x: np.sum(~np.isnan(x), axis=0))

In [ ]:
grouped_all_trials_df["sem_power"] = grouped_all_trials_df.apply(lambda x: x["std_power"] / np.sqrt(x["n_trials"]), axis=1)

In [ ]:
grouped_all_trials_df["competition_closeness"].unique()

# Drawing plots all together

In [ ]:
region_df

In [ ]:
low_freq = 0
high_freq = 12
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 0 to 5cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)  
    plt.yscale("log")
    plt.ylim(0.01, 0.1) 


    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} {}".format(row["competition_closeness"], row["trial_or_baseline"]), color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]], \
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)
            
            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
            alpha=0.1, color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]])
        
        except Exception as e: 
            print(e)
            continue
    
    # Create a legend outside of the plot
    legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Save the plot along with the legend outside of it   
    plt.savefig("./proc/velocity_parsing/plot_region_line_competitioncloseness_baselinetrial/\
    rce_lower_velocity_lfp_power_competitioncloseness_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]), bbox_extra_artists=(legend,), bbox_inches='tight')

    # Display the plot
    plt.show()


In [ ]:
low_freq = 12
high_freq = 30
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 0 to 5cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.002, 0.015) 

    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} {}".format(row["competition_closeness"], row["trial_or_baseline"]), color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]], \
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)
            
            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
            alpha=0.1, color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]])
        
        except Exception as e: 
            print(e)
            continue
            
    # Create a legend outside of the plot
    legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Save the plot along with the legend outside of it   
    plt.savefig("./proc/velocity_parsing/plot_region_line_competitioncloseness_baselinetrial/\
    rce_lower_velocity_lfp_power_competitioncloseness_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]), bbox_extra_artists=(legend,), bbox_inches='tight')

    # Display the plot
    plt.show()

In [ ]:
low_freq = 30
high_freq = 90
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 0 to 5cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.0003, 0.005) 

    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} {}".format(row["competition_closeness"], row["trial_or_baseline"]), color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]], \
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)
            
            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
            alpha=0.1, color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]])
        
        except Exception as e: 
            print(e)
            continue
    # Create a legend outside of the plot
    legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Save the plot along with the legend outside of it   
    plt.savefig("./proc/velocity_parsing/plot_region_line_competitioncloseness_baselinetrial/\
    rce_lower_velocity_lfp_power_competitioncloseness_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]), bbox_extra_artists=(legend,), bbox_inches='tight')

    # Display the plot
    plt.show()

## Plotting all the higher velocities together

In [ ]:
speed_filtered_all_trials_df = exploded_channel_map_and_all_trials_df[exploded_channel_map_and_all_trials_df["binned_current_thorax_velocity"] == 1]

In [ ]:
grouped_all_trials_df = speed_filtered_all_trials_df.groupby(['brain_region', 'competition_closeness', 'trial_or_baseline']).agg({'power': lambda x: np.vstack(x.tolist())}).reset_index()

In [ ]:
grouped_all_trials_df["mean_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanmean(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["std_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanstd(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["n_trials"] = grouped_all_trials_df["power"].apply(lambda x: np.sum(~np.isnan(x), axis=0))

In [ ]:
grouped_all_trials_df["sem_power"] = grouped_all_trials_df.apply(lambda x: x["std_power"] / np.sqrt(x["n_trials"]), axis=1)

# Drawing plots all together

In [ ]:
low_freq = 0
high_freq = 12
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 5cm/s to 10cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.01, 0.25) 

    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} {}".format(row["competition_closeness"], row["trial_or_baseline"]), color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]], \
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)
            
            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
            alpha=0.1, color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]])
        
        except Exception as e: 
            print(e)
            continue
    # Create a legend outside of the plot
    legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Save the plot along with the legend outside of it   
    plt.savefig("./proc/velocity_parsing/plot_region_line_competitioncloseness_baselinetrial/\
    rce_higher_velocity_lfp_power_competitioncloseness_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]), bbox_extra_artists=(legend,), bbox_inches='tight')

    # Display the plot
    plt.show()

In [ ]:
low_freq = 12
high_freq = 30
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 5cm/s to 10cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.002, 0.015) 

    
    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} {}".format(row["competition_closeness"], row["trial_or_baseline"]), color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]], \
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)
            
            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
            alpha=0.1, color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]])
        
        except Exception as e: 
            print(e)
            continue
    # Create a legend outside of the plot
    legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Save the plot along with the legend outside of it   
    plt.savefig("./proc/velocity_parsing/plot_region_line_competitioncloseness_baselinetrial/\
    rce_higher_velocity_lfp_power_competitioncloseness_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]), bbox_extra_artists=(legend,), bbox_inches='tight')

    # Display the plot
    plt.show()

In [ ]:
low_freq = 12
high_freq = 30
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 5cm/s to 10cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.002, 0.015) 
    
    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} {}".format(row["competition_closeness"], row["trial_or_baseline"]), color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]], \
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)
            
            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
            alpha=0.1, color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]])
        
        except Exception as e: 
            print(e)
            continue
    # Create a legend outside of the plot
    legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Save the plot along with the legend outside of it   
    plt.savefig("./proc/velocity_parsing/plot_region_line_competitioncloseness_baselinetrial/\
    rce_higher_velocity_lfp_power_competitioncloseness_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]), bbox_extra_artists=(legend,), bbox_inches='tight')

    # Display the plot
    plt.show()

In [ ]:
low_freq = 30
high_freq = 90
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 5cm/s to 10cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.0003, 0.005) 
    
    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} {}".format(row["competition_closeness"], row["trial_or_baseline"]), color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]], \
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)
            
            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
            alpha=0.1, color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]])
        
        except Exception as e: 
            print(e)
            continue
            
    # Create a legend outside of the plot
    legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Save the plot along with the legend outside of it   
    plt.savefig("./proc/velocity_parsing/plot_region_line_competitioncloseness_baselinetrial/\
    rce_higher_velocity_lfp_power_competitioncloseness_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]), bbox_extra_artists=(legend,), bbox_inches='tight')

    # Display the plot
    plt.show()

In [ ]:
exploded_channel_map_and_all_trials_df

In [ ]:
exploded_channel_map_and_all_trials_df.columns

In [ ]:
exploded_channel_map_and_all_trials_df["current_thorax_velocity"]

In [ ]:
melted_channel_map_and_all_trials_df["subject_id"].unique()

# Drawing Spectogram

In [ ]:
exploded_channel_map_and_all_trials_df["power"].iloc[0].shape

In [ ]:
exploded_channel_map_and_all_trials_df["brain_region"]

In [ ]:
exploded_channel_map_and_all_trials_df[""]

In [ ]:
grouped_all_trials_df

In [ ]:
raise ValueError()

In [ ]:
exploded_channel_map_and_all_trials_df

In [ ]:
exploded_channel_map_and_all_trials_df["recording_file"].unique()

In [ ]:
exploded_channel_map_and_all_trials_df["subject_id"].unique()